In [37]:
import torch
import matplotlib.pyplot as plt
import matplotlib.cm as CM
from tqdm import tqdm

from model import CSRNet
from dataset import CrowdDataset

In [52]:
def cal_mae(img_root,gt_dmap_root,model_param_path):
    '''
    Calculate the MAE of the test data.
    img_root: the root of test image data.
    gt_dmap_root: the root of test ground truth density-map data.
    model_param_path: the path of specific mcnn parameters.
    '''
    #device=torch.device("cuda")
    cfg = Config()
    model=CSRNet()
    #model.load_state_dict(torch.load(model_param_path))
    torch.load(model_param_path, map_location=lambda storage, loc: storage)
    model.to(cfg.device)
    dataset=CrowdDataset(img_root,gt_dmap_root,8)
    dataloader=torch.utils.data.DataLoader(dataset,batch_size=1,shuffle=False)
    model.eval()
    mae=0
    with torch.no_grad():
        for i,(img,gt_dmap) in enumerate(tqdm(dataloader)):
            img=img.to(device)
            gt_dmap=gt_dmap.to(device)
            # forward propagation
            et_dmap=model(img)
            mae+=abs(et_dmap.data.sum()-gt_dmap.data.sum()).item()
            del img,gt_dmap,et_dmap

    print("model_param_path:"+model_param_path+" mae:"+str(mae/len(dataloader)))

In [63]:
def estimate_density_map(img_root,gt_dmap_root,model_param_path,index):
    '''
    Show one estimated density-map.
    img_root: the root of test image data.
    gt_dmap_root: the root of test ground truth density-map data.
    model_param_path: the path of specific mcnn parameters.
    index: the order of the test image in test dataset.
    '''
    #device=torch.device("cuda")
    cfg = Config()
    model=CSRNet()
    #model.load_state_dict(torch.load(model_param_path))
    torch.load(model_param_path, map_location=lambda storage, loc: storage)
    model.to(cfg.device)
    dataset=CrowdDataset(img_root,gt_dmap_root,8)
    dataloader=torch.utils.data.DataLoader(dataset,batch_size=1,shuffle=False)
    model.eval()
    for i,(img,gt_dmap) in enumerate(dataloader):
        if i==index:
            img=img.to(device)
            gt_dmap=gt_dmap.to(device)
            # forward propagation
            et_dmap=model(img).detach()
            et_dmap=et_dmap.squeeze(0).squeeze(0).cpu().numpy()
            print(et_dmap.shape)
            plt.imshow(et_dmap,cmap=CM.jet)
            break

In [80]:
torch.backends.cudnn.enabled=False
img_root='./data/part_A_final/test'
gt_dmap_root='./data/part_A_final/test'
model_param_path='./checkpoints/shaghai_tech_a_best.pth'
cal_mae(img_root,gt_dmap_root,model_param_path)
#estimate_density_map(img_root,gt_dmap_root,model_param_path,3) 

FileNotFoundError: [Errno 2] No such file or directory: './data/part_A_final/test/./data/part_A_final/test_data/images'